# Introduction and Setup

# Irish Landmark Recognition - Web Application Development

## Objectives
This notebook creates an interactive web application for our Irish landmark recognition model:
- Streamlit-based user interface
- Image upload and preprocessing
- Real-time predictions with confidence scores
- Interactive visualizations
- Model information and statistics

## Features
- Image upload interface
- Real-time landmark predictions
- Confidence score visualization
-  Mobile-friendly responsive design
-  Educational landmark information

In [1]:
# CELL 2: Import Libraries and Setup
import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import json
from pathlib import Path
import io
import base64

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print(" Libraries imported successfully!")
print(" Ready to create Streamlit web application")

 Libraries imported successfully!
 Ready to create Streamlit web application


In [2]:
# CELL 3: Model Loading Functions
class IrishLandmarkClassifier(nn.Module):
    """Model class for landmark classification"""
    
    def __init__(self, num_classes=6, model_name='resnet18'):
        super(IrishLandmarkClassifier, self).__init__()
        
        if model_name == 'resnet18':
            self.backbone = models.resnet18(pretrained=False)
            self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)
        elif model_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=False)
            self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)
        elif model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=False)
            self.backbone.classifier[1] = nn.Linear(
                self.backbone.classifier[1].in_features, 
                num_classes
            )
    
    def forward(self, x):
        return self.backbone(x)

@st.cache_resource
def load_model_and_config():
    """Load model and configuration (cached for performance)"""
    
    model_path = "models/irish_landmarks_resnet18.pth"
    device = torch.device("cpu")  # Use CPU for web app
    
    try:
        # Load checkpoint
        checkpoint = torch.load(model_path, map_location=device)
        
        # Extract configuration
        model_name = checkpoint.get('model_name', 'resnet18')
        num_classes = checkpoint.get('num_classes', 6)
        class_names = checkpoint.get('class_names', [])
        
        # Initialize model
        model = IrishLandmarkClassifier(num_classes=num_classes, model_name=model_name)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        return model, {
            'class_names': class_names,
            'num_classes': num_classes,
            'model_name': model_name,
            'best_accuracy': checkpoint.get('training_history', {}).get('best_val_acc', 0.85)
        }
        
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None, None


In [3]:
# CELL 4: Image Preprocessing Functions
def get_image_transforms():
    """Get image preprocessing transforms"""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def preprocess_image(image):
    """Preprocess uploaded image for prediction"""
    transform = get_image_transforms()
    
    # Convert to RGB if needed
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Apply transforms
    image_tensor = transform(image).unsqueeze(0)
    return image_tensor

def predict_landmark(model, image_tensor):
    """Make prediction on preprocessed image"""
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        confidence, predicted = torch.max(probabilities, 0)
    
    return predicted.item(), confidence.item(), probabilities.numpy()


In [4]:
# CELL 5: Landmark Information Database
@st.cache_data
def get_landmark_info():
    """Get detailed information about each landmark"""
    return {
        'Cliffs of Moher': {
            'location': 'County Clare',
            'description': 'The Cliffs of Moher are sea cliffs located at the southwestern edge of the Burren region in County Clare, Ireland. They rise 120 metres above the Atlantic Ocean at Hag\'s Head, and reach their maximum height of 214 metres just north of O\'Brien\'s Tower.',
            'fun_facts': [
                'Featured in movies like Harry Potter and The Princess Bride',
                'Visited by over 1.5 million people annually',
                'Home to over 30,000 breeding pairs of seabirds',
                'The cliffs stretch for 14 kilometers along the coast'
            ],
            'best_time_to_visit': 'May to September for clearest weather',
            'unesco_status': 'Not a UNESCO site, but part of the Burren and Cliffs of Moher UNESCO Global Geopark'
        },
        "Giant's Causeway": {
            'location': 'County Antrim, Northern Ireland',
            'description': 'The Giant\'s Causeway is an area of about 40,000 interlocking basalt columns, the result of an ancient volcanic fissure eruption. It is located in County Antrim on the north coast of Northern Ireland.',
            'fun_facts': [
                'Made up of approximately 40,000 hexagonal basalt columns',
                'Formed 50-60 million years ago during volcanic activity',
                'Legend says it was built by giant Finn MacCool',
                'The tallest columns are about 12 metres high'
            ],
            'best_time_to_visit': 'Early morning or late afternoon to avoid crowds',
            'unesco_status': 'UNESCO World Heritage Site since 1986'
        },
        'Ring of Kerry': {
            'location': 'County Kerry',
            'description': 'The Ring of Kerry is a 179-kilometre long circular tourist route in County Kerry, southwestern Ireland. The route encircles the Iveragh Peninsula and offers stunning views of mountains, lakes, and the Atlantic coastline.',
            'fun_facts': [
                'One of Ireland\'s most popular scenic drives',
                'Takes approximately 4-6 hours to drive without stops',
                'Passes through charming towns like Killarney and Kenmare',
                'Offers views of the MacGillycuddy\'s Reeks mountain range'
            ],
            'best_time_to_visit': 'May to September for best weather and longest days',
            'unesco_status': 'Not a UNESCO site, but contains Skellig Michael UNESCO site nearby'
        },
        'Dublin Castle': {
            'location': 'Dublin City Center',
            'description': 'Dublin Castle is a major Irish government complex, conference centre, and tourist attraction. It was chosen for its position at the highest point of central Dublin, on a ridge to the south of the River Liffey.',
            'fun_facts': [
                'Built in 1204 on the site of a Viking settlement',
                'Served as the seat of British rule in Ireland for over 700 years',
                'Now hosts Irish presidential inaugurations',
                'Houses the Chester Beatty Library with rare manuscripts'
            ],
            'best_time_to_visit': 'Year-round, indoor attraction with guided tours',
            'unesco_status': 'Not a UNESCO site, but of significant historical importance'
        },
        'Killarney National Park': {
            'location': 'County Kerry',
            'description': 'Killarney National Park is located beside the town of Killarney, County Kerry, Ireland. It was the first national park established in Ireland, created when the Muckross Estate was donated to the Irish Free State in 1932.',
            'fun_facts': [
                'Ireland\'s first national park, established in 1932',
                'Home to the only native herd of red deer in Ireland',
                'Contains three glacial lakes: Lough Leane, Muckross Lake, and Upper Lake',
                'Covers over 10,000 hectares of diverse landscapes'
            ],
            'best_time_to_visit': 'April to October for hiking and outdoor activities',
            'unesco_status': 'UNESCO Biosphere Reserve'
        },
        'Rock of Cashel': {
            'location': 'County Tipperary',
            'description': 'The Rock of Cashel, also known as Cashel of the Kings and St. Patrick\'s Rock, is a historic site located at Cashel, County Tipperary. The Rock of Cashel was the traditional seat of the kings of Munster for several hundred years prior to the Norman invasion.',
            'fun_facts': [
                'Also known as the "Cashel of the Kings"',
                'St. Patrick baptized King Aengus here in 450 AD',
                'The round tower is 28 metres tall and over 1,000 years old',
                'Offers panoramic views of the Tipperary countryside'
            ],
            'best_time_to_visit': 'Year-round, but spring and summer offer the best photography light',
            'unesco_status': 'Not a UNESCO site, but on the tentative list for World Heritage status'
        }
    }

2025-08-10 15:00:21.970 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [5]:
# CELL 6: Streamlit App Configuration
def configure_streamlit_app():
    """Configure Streamlit app settings"""
    st.set_page_config(
        page_title="Irish Landmark Recognition",
        page_icon="🏰",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # Custom CSS for better styling
    st.markdown("""
    <style>
    .main-header {
        text-align: center;
        color: #1f4e79;
        font-size: 2.5rem;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .subtitle {
        text-align: center;
        color: #666;
        font-size: 1.2rem;
        margin-bottom: 2rem;
    }
    .prediction-box {
        background-color: #f0f8ff;
        padding: 1rem;
        border-radius: 10px;
        border-left: 5px solid #1f4e79;
        margin: 1rem 0;
    }
    .landmark-info {
        background-color: #f9f9f9;
        padding: 1rem;
        border-radius: 10px;
        margin: 1rem 0;
    }
    </style>
    """, unsafe_allow_html=True)

In [6]:
# CELL 7: Main Streamlit Application
def create_streamlit_app():
    """Create the main Streamlit application"""
    
    # Configure app
    configure_streamlit_app()
    
    # Load model and config
    model, config = load_model_and_config()
    landmark_info = get_landmark_info()
    
    # Header
    st.markdown('<h1 class="main-header">🏰 Irish Landmark Recognition</h1>', unsafe_allow_html=True)
    st.markdown('<p class="subtitle">Upload a photo of an Irish landmark and let AI identify it!</p>', unsafe_allow_html=True)
    
    # Sidebar with app information
    with st.sidebar:
        st.header("📱 About This App")
        st.markdown("""
        This AI-powered application can recognize **6 famous Irish landmarks**:
        
         **Cliffs of Moher** (County Clare)  
         **Giant's Causeway** (County Antrim)  
         **Ring of Kerry** (County Kerry)  
         **Dublin Castle** (Dublin)  
         **Killarney National Park** (County Kerry)  
         **Rock of Cashel** (County Tipperary)
        """)
        
        if config:
            st.header(" Model Info")
            st.metric("Model Accuracy", f"{config['best_accuracy']:.1%}")
            st.metric("Architecture", config['model_name'].upper())
            st.metric("Classes", config['num_classes'])
        
        st.header("💡 Tips for Best Results")
        st.markdown("""
        - Upload clear, well-lit photos
        - Ensure the landmark is the main subject
        - Avoid heavily filtered or edited images
        - JPG, JPEG, and PNG formats supported
        """)
    
    # Main content area
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.header(" Upload Your Image")
        
        uploaded_file = st.file_uploader(
            "Choose an image of an Irish landmark",
            type=['jpg', 'jpeg', 'png'],
            help="Upload a clear photo of one of the supported Irish landmarks"
        )
        
        if uploaded_file is not None:
            # Display uploaded image
            image = Image.open(uploaded_file)
            st.image(image, caption="Uploaded Image", use_column_width=True)
            
            if model is not None:
                # Make prediction
                with st.spinner(' Analyzing image...'):
                    image_tensor = preprocess_image(image)
                    predicted_idx, confidence, probabilities = predict_landmark(model, image_tensor)
                
                # Display prediction results
                predicted_landmark = config['class_names'][predicted_idx]
                
                st.markdown('<div class="prediction-box">', unsafe_allow_html=True)
                st.subheader(" Prediction Results")
                
                # Main prediction with confidence
                col_pred1, col_pred2 = st.columns([3, 1])
                with col_pred1:
                    st.success(f"**Predicted Landmark:** {predicted_landmark}")
                with col_pred2:
                    st.metric("Confidence", f"{confidence:.1%}")
                
                st.markdown('</div>', unsafe_allow_html=True)
                
                # Confidence visualization
                st.subheader(" Prediction Confidence")
                
                # Create confidence chart
                confidence_df = pd.DataFrame({
                    'Landmark': config['class_names'],
                    'Confidence': probabilities * 100
                })
                confidence_df = confidence_df.sort_values('Confidence', ascending=True)
                
                fig = px.bar(
                    confidence_df,
                    x='Confidence',
                    y='Landmark',
                    orientation='h',
                    title='Confidence Scores for All Landmarks',
                    color='Confidence',
                    color_continuous_scale='viridis',
                    text='Confidence'
                )
                
                fig.update_traces(texttemplate='%{text:.1f}%', textposition='inside')
                fig.update_layout(
                    height=400,
                    xaxis_title="Confidence (%)",
                    yaxis_title="Landmark",
                    showlegend=False
                )
                
                st.plotly_chart(fig, use_container_width=True)
                
            else:
                st.error(" Model not available. Please ensure the model file exists.")
    
    with col2:
        if uploaded_file is not None and model is not None:
            # Display landmark information
            if predicted_landmark in landmark_info:
                info = landmark_info[predicted_landmark]
                
                st.markdown('<div class="landmark-info">', unsafe_allow_html=True)
                st.subheader(f" About {predicted_landmark}")
                
                st.write(f"** Location:** {info['location']}")
                st.write(f"** Description:** {info['description']}")
                
                st.write("** Fun Facts:**")
                for fact in info['fun_facts']:
                    st.write(f"• {fact}")
                
                st.write(f"** Best Time to Visit:** {info['best_time_to_visit']}")
                st.write(f"** UNESCO Status:** {info['unesco_status']}")
                
                st.markdown('</div>', unsafe_allow_html=True)
            
            # Additional predictions section
            st.subheader("🔍 Alternative Predictions")
            
            # Show top 3 predictions
            top_indices = np.argsort(probabilities)[::-1][:3]
            
            for i, idx in enumerate(top_indices):
                landmark = config['class_names'][idx]
                conf = probabilities[idx]
                
                if i == 0:
                    st.success(f"1 {landmark}: {conf:.1%}")
                elif i == 1:
                    st.info(f"2 {landmark}: {conf:.1%}")
                else:
                    st.warning(f"3 {landmark}: {conf:.1%}")
        
        else:
            # Show example images when no upload
            st.subheader(" Example Landmarks")
            st.markdown("""
            Here are the landmarks our AI can recognize:
            
             **Cliffs of Moher** - Dramatic Atlantic cliffs  
             **Giant's Causeway** - Unique hexagonal stones  
             **Ring of Kerry** - Scenic circular route  
             **Dublin Castle** - Historic city center castle  
             **Killarney National Park** - Lakes and mountains  
             **Rock of Cashel** - Medieval religious site  
            """)
    
    # Footer
    st.markdown("---")
    col_footer1, col_footer2, col_footer3 = st.columns(3)
    
    with col_footer1:
        st.markdown("** Built With:**")
        st.markdown("PyTorch • Streamlit • ResNet-18")
    
    with col_footer2:
        st.markdown("** Performance:**")
        if config:
            st.markdown(f"Accuracy: {config['best_accuracy']:.1%} • Real-time inference")
        else:
            st.markdown("Model loading...")
    
    with col_footer3:
        st.markdown("**🎓 Academic Project:**")
        st.markdown("National College of Ireland • Deep Learning Course")


In [7]:
# CELL 8: Create App File for Deployment
def create_streamlit_app_file():
    """Create the standalone app.py file for deployment"""
    
    app_code = '''import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms, models
import numpy as np
import pandas as pd
from PIL import Image
import plotly.express as px
import json
from pathlib import Path

# Configure Streamlit
st.set_page_config(
    page_title="Irish Landmark Recognition",
    page_icon="Castle",  # Using text instead of emoji
    layout="wide",
    initial_sidebar_state="expanded"
)

# Model class
class IrishLandmarkClassifier(nn.Module):
    def __init__(self, num_classes=6, model_name='resnet18'):
        super(IrishLandmarkClassifier, self).__init__()
        
        if model_name == 'resnet18':
            self.backbone = models.resnet18(pretrained=False)
            self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.backbone(x)

@st.cache_resource
def load_model_and_config():
    """Load model and configuration"""
    try:
        checkpoint = torch.load("models/irish_landmarks_resnet18.pth", map_location="cpu")
        
        model = IrishLandmarkClassifier()
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        return model, checkpoint.get('class_names', [])
    except:
        return None, ['Cliffs of Moher', "Giant's Causeway", 'Ring of Kerry', 
                     'Dublin Castle', 'Killarney National Park', 'Rock of Cashel']

def preprocess_image(image):
    """Preprocess image for prediction"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    return transform(image).unsqueeze(0)

def predict_landmark(model, image_tensor):
    """Make prediction"""
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        confidence, predicted = torch.max(probabilities, 0)
    
    return predicted.item(), confidence.item(), probabilities.numpy()

# Main app
def main():
    st.title(" Irish Landmark Recognition")
    st.markdown("Upload a photo of an Irish landmark and let AI identify it!")
    
    model, class_names = load_model_and_config()
    
    uploaded_file = st.file_uploader("Choose an image", type=['jpg', 'jpeg', 'png'])
    
    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)
        
        if model is not None:
            image_tensor = preprocess_image(image)
            predicted_idx, confidence, probabilities = predict_landmark(model, image_tensor)
            
            predicted_landmark = class_names[predicted_idx]
            
            st.success(f"Predicted: {predicted_landmark}")
            st.info(f"Confidence: {confidence:.1%}")
            
            # Confidence chart
            confidence_df = pd.DataFrame({
                'Landmark': class_names,
                'Confidence': probabilities * 100
            })
            
            fig = px.bar(confidence_df, x='Landmark', y='Confidence', 
                        title='Confidence Scores')
            st.plotly_chart(fig)

if __name__ == "__main__":
    main()
'''
    
    # Save app file
    app_file_path = Path("app.py")
    with open(app_file_path, 'w') as f:
        f.write(app_code)
    
    print(f" Streamlit app file created: {app_file_path}")
    return app_file_path


In [8]:
# CELL 9: Create Requirements File
def create_requirements_file():
    """Create requirements.txt for deployment"""
    
    requirements = [
        "streamlit>=1.25.0",
        "torch>=1.12.0",
        "torchvision>=0.13.0",
        "numpy>=1.21.0",
        "pandas>=1.5.0",
        "pillow>=9.0.0",
        "plotly>=5.0.0",
        "pathlib",
    ]
    
    requirements_path = Path("requirements.txt")
    with open(requirements_path, 'w') as f:
        f.write('\n'.join(requirements))
    
    print(f" Requirements file created: {requirements_path}")
    return requirements_path

In [9]:
# CELL 10: Run the Application
if __name__ == "__main__":
    print(" Creating Streamlit application...")
    
    # Option 1: Run in notebook (for development)
    print("\n To run the app in development mode:")
    print("1. Uncomment the line below and run this cell")
    print("2. Or run the app in terminal: streamlit run app.py")
    
    # Uncomment the next line to run the app directly in the notebook
    # create_streamlit_app()
    
    # Option 2: Create standalone files for deployment
    print("\n Creating deployment files...")
    app_file = create_streamlit_app_file()
    req_file = create_requirements_file()
    
    print("\n Web application ready!")
    print("=" * 40)
    print(" Streamlit app created successfully")
    print(f" App file: {app_file}")
    print(f" Requirements: {req_file}")
    print("\n To run the application:")
    print("1. Ensure your model file exists: models/irish_landmarks_resnet18.pth")
    print("2. Install requirements: pip install -r requirements.txt")
    print("3. Run the app: streamlit run app.py")
    print("4. Open your browser to the provided URL (usually http://localhost:8501)")
    
    print("\n DEPLOYMENT OPTIONS:")
    print("• Local: Run on your computer using the commands above")
    print("• Streamlit Cloud: Push to GitHub and deploy via share.streamlit.io")
    print("• Heroku: Deploy using git and Heroku CLI")
    print("• Google Cloud: Deploy on Cloud Run or App Engine")

 Creating Streamlit application...

 To run the app in development mode:
1. Uncomment the line below and run this cell
2. Or run the app in terminal: streamlit run app.py

 Creating deployment files...
 Streamlit app file created: app.py
 Requirements file created: requirements.txt

 Web application ready!
 Streamlit app created successfully
 App file: app.py
 Requirements: requirements.txt

 To run the application:
1. Ensure your model file exists: models/irish_landmarks_resnet18.pth
2. Install requirements: pip install -r requirements.txt
3. Run the app: streamlit run app.py
4. Open your browser to the provided URL (usually http://localhost:8501)

 DEPLOYMENT OPTIONS:
• Local: Run on your computer using the commands above
• Streamlit Cloud: Push to GitHub and deploy via share.streamlit.io
• Heroku: Deploy using git and Heroku CLI
• Google Cloud: Deploy on Cloud Run or App Engine


In [10]:
# CELL 11: Testing the Application Components
def test_app_components():
    """Test individual components of the application"""
    
    print(" Testing application components...")
    
    # Test model loading
    try:
        model, config = load_model_and_config()
        if model is not None:
            print(" Model loading: SUCCESS")
        else:
            print(" Model loading: Model file not found (expected for first run)")
    except Exception as e:
        print(f" Model loading: FAILED - {e}")
    
    # Test image preprocessing
    try:
        # Create test image
        test_image = Image.new('RGB', (300, 300), color='blue')
        processed = preprocess_image(test_image)
        assert processed.shape == (1, 3, 224, 224)
        print(" Image preprocessing: SUCCESS")
    except Exception as e:
        print(f" Image preprocessing: FAILED - {e}")
    
    # Test landmark info loading
    try:
        landmark_info = get_landmark_info()
        assert len(landmark_info) == 6
        print(" Landmark info loading: SUCCESS")
    except Exception as e:
        print(f" Landmark info loading: FAILED - {e}")
    
    print("\n Component testing complete!")

# Run tests
test_app_components()

print("\n WEB APPLICATION DEVELOPMENT COMPLETE!")
print("=" * 50)
print(" Interactive Streamlit application created")
print(" Mobile-friendly responsive design")
print(" Real-time landmark predictions")
print(" Interactive confidence visualizations")
print("ℹ Educational landmark information")
print(" Ready for deployment!")
print("\nYour Irish Landmark Recognition project is now complete! 🇮🇪")

2025-08-10 15:00:31.556 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.705 
  command:

    streamlit run c:\Users\aryam\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-10 15:00:31.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


 Testing application components...


2025-08-10 15:00:31.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.836 No runtime found, using MemoryCacheStorageManager
2025-08-10 15:00:31.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 15:00:31.839 Thread 'MainThread':

 Model loading: SUCCESS
 Image preprocessing: SUCCESS
 Landmark info loading: SUCCESS

 Component testing complete!

 WEB APPLICATION DEVELOPMENT COMPLETE!
 Interactive Streamlit application created
 Mobile-friendly responsive design
 Real-time landmark predictions
 Interactive confidence visualizations
ℹ Educational landmark information
 Ready for deployment!

Your Irish Landmark Recognition project is now complete! 🇮🇪
